In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F
session = get_active_session()


In [ ]:
table_name = "WINE_QUALITY_DB.PUBLIC.WINE_QUALITY_TABLE"

# Create a Snowpark DataFrame (this doesn't download data, it just points to it)
df = session.table(table_name)
df.show(5)

Feature Engineering

In [ ]:
print(df.columns)

In [ ]:
print(df.describe())

In [ ]:
print("Data Schema:")
for field in df.schema.fields:
    print(f"{field.name}: {field.datatype}")

In [ ]:
df.describe().show()

In [ ]:
total_rows = df.count()
print(f"Total Rows: {total_rows}")

# Loop through columns to find missing values
# (Snowpark doesn't have a one-line isnull().sum() like pandas)
for col_name in df.columns:
    null_count = df.filter(F.col(col_name).is_null()).count()
    if null_count > 0:
        print(f"Column {col_name} has {null_count} nulls")
    else:
        print(f"Column {col_name} has no nulls")

In [ ]:
duplicates_count = total_rows - df.distinct().count()
print(f"Duplicate records found: {duplicates_count}")

In [ ]:
df = df.drop_duplicates()

print(f"New row count after dropping duplicates: {df.count()}")

In [ ]:
# Cell 4: Visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Convert Snowpark DF to Pandas DF for Visualization
# This brings the data from the server into the notebook memory
pdf = df.to_pandas()

# 2. Correlation Matrix
plt.figure(figsize=(10,6))
# Using 'numeric_only=True' is safer to avoid errors with strings
sns.heatmap(pdf.corr(numeric_only=True), annot=True, cmap="coolwarm")
plt.title("Correlation Heatmap")
plt.show()

# 3. Target Balance (Imbalanced Dataset Check)
plt.figure(figsize=(6,4))
pdf['QUALITY'].value_counts().sort_index().plot(kind='bar')
plt.xlabel("Wine Quality")
plt.ylabel("Count")
plt.title("Distribution of Wine Quality")
plt.show()

In [ ]:
# Cell 5: Advanced Plots (Distributions & Relations)

# 1. Histograms for all columns
# We use the pandas dataframe 'pdf' we created in the previous cell
pdf.hist(bins=15, figsize=(15, 10), layout=(4, 3))
plt.tight_layout()
plt.show()

# 2. Box Plot (Alcohol vs Quality)
plt.figure(figsize=(8,5))
sns.boxplot(x='QUALITY', y='ALCOHOL', data=pdf)
plt.title("Alcohol Content vs Wine Quality")
plt.show()

# 3. Scatter Plot (Alcohol vs pH)
plt.figure(figsize=(8,5))
sns.scatterplot(x='ALCOHOL', y='PH', hue='QUALITY', palette='viridis', data=pdf)
plt.title("Alcohol vs pH (Colored by Quality)")
plt.show()

In [ ]:
df.show(5)

In [ ]:
df.columns

In [ ]:
# Cell 1: Data Prep & Engineering
from snowflake.snowpark.window import Window
import snowflake.snowpark.functions as F

# 1. Add WINE_ID using a Window Function
# We order by a constant to treat the whole table as one group for row numbering
window_spec = Window.order_by(F.lit(1))
df = df.with_column("WINE_ID", F.row_number().over(window_spec))

# 2. Feature Engineering
# Create Target: IS_GOOD_QUALITY (1 if Quality >= 7, else 0)
# Create Feature: ACIDITY_RATIO
df = df.with_column("IS_GOOD_QUALITY", F.when(F.col("QUALITY") >= 7, 1).otherwise(0)) \
       .with_column("ACIDITY_RATIO", F.col("FIXED_ACIDITY") / F.col("VOLATILE_ACIDITY"))

# 3. Create 'final_df' with all necessary columns
# We keep WINE_ID, Target, and all Input Features
final_df = df.select(
    "WINE_ID", 
    "ALCOHOL", 
    "SULPHATES", 
    "ACIDITY_RATIO", 
    "IS_GOOD_QUALITY"
)

print("Data Prepared Successfully.")
final_df.show(5)

In [ ]:
# Cell 2: Register Feature View (v2)
from snowflake.ml.feature_store import FeatureStore, FeatureView, Entity, CreationMode

# 1. Initialize Feature Store
fs = FeatureStore(
    session=session, 
    database=session.get_current_database(), 
    name=session.get_current_schema(), 
    default_warehouse="COMPUTE_WH", 
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST
)

# 2. Register Entity
entity = Entity(name="WINE_ENTITY", join_keys=["WINE_ID"])
fs.register_entity(entity)

# 3. Prepare Features DataFrame (Inputs ONLY)
# CRITICAL: We select only the ID and the Features. We LEAVE OUT 'IS_GOOD_QUALITY'.
features_only_df = final_df.select("WINE_ID", "ALCOHOL", "SULPHATES", "ACIDITY_RATIO")

# 4. Register Feature View v2
fv_v2 = FeatureView(
    name="WINE_FEATURES_FV", 
    entities=[entity], 
    feature_df=features_only_df, 
    refresh_freq="1 minute",
    desc="v2: Features ONLY (Target excluded)"
)

# Register (Overwrite if exists)
fs.register_feature_view(feature_view=fv_v2, version="v2", block=True, overwrite=True)
print("Feature View v2 registered successfully (Target column removed).")

In [ ]:
# Cell 3: Retrieve Data & Train Model
from snowflake.ml.modeling.linear_model import LogisticRegression
from snowflake.ml.modeling.metrics import accuracy_score

# 1. Retrieve the Feature View Freshly
# This prevents the 'NoneType' connection error
fv_fresh = fs.get_feature_view(name="WINE_FEATURES_FV", version="v2")
print(f"Using Feature View: {fv_fresh.name}, Version: {fv_fresh.version}")

# 2. Generate Training Dataset
# Spine = ID + Target ('IS_GOOD_QUALITY')
# Features = ID + Inputs ('ALCOHOL', etc.)
# The join happens automatically on WINE_ID
training_ds = fs.generate_dataset(
    name="WINE_TRAINING_SET",
    version="v5", 
    spine_df=final_df.select("WINE_ID", "IS_GOOD_QUALITY"), 
    features=[fv_fresh], 
    spine_timestamp_col=None
)

# 3. Train Model
train_sdf = training_ds.read.to_snowpark_dataframe()

lr = LogisticRegression(
    label_cols=["IS_GOOD_QUALITY"], 
    output_cols=["PREDICTION"], 
    input_cols=["ALCOHOL", "SULPHATES", "ACIDITY_RATIO"] 
)

print("Training Logistic Regression model...")
model = lr.fit(train_sdf)

# 4. Evaluate
predictions = model.predict(train_sdf)
acc = accuracy_score(
    df=predictions, 
    y_true_col_names=["IS_GOOD_QUALITY"], 
    y_pred_col_names=["PREDICTION"]
)

print(f"Model Training Complete.")
print(f"Accuracy Score: {acc:.2f}")